In [1]:
from pathlib import Path

import pdfplumber
import re
import unicodedata
from pprint import pprint

import sys
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

In [2]:
from pdf_processing import *

TODO: 
 - check more PDFs for differences in format
 - write documentation
 - locations not included after the date (might need to be taken care of manually)
 - maybe use translate() instead of pdf.replace()



DONE: 
 - Q&As / interviews
     - function to count the number of occurrences of "Obama:", "Question:", etc.
 
 
 - footnotes (like in People_of_Greece.pdf and Senate_Floor_Immigration_Reform.pdf)
     - seems to work now
     
     
 - locations with multiple comma's (ex. when Washington, D.C. is included) --> might be better to store location as one elements instead of splitting it up, the location can be cleaned up afterwards
     - location is not split up anymore, but kept as a single element (will be easier to clean up afterwards)

In [3]:
# pdfs that have these in the title might have to be excluded: interview, presser, Press_Conference, press, meeting, 
# debate, hall (maybe)
multiple_people_talking = ['Barack_Obama_-_Al-Arabiya_Interview.pdf', 'CGI_2013.pdf', 
                           'Midterm_Elections_Presser_2014.pdf', 'YSEALI_Fellows.pdf', 
                           'PM_Abe_of_Japan_Joint_Presser.pdf', 'Paris_Press_Conference_2015.pdf',
                           'Gun_Violence_Presser.pdf', 'Associated_Press_Luncheon.pdf', 'Final_Press_Conference.pdf',
                           'CNN_Guns_in_America_Town_Hall.pdf', 'START_Treaty_Presser.pdf', 
                           'Midterm_Elections_Presser_2010.pdf', 'Joint_Presser_with_President_Benigno_Aquino.pdf',
                           'Worldwide_Troop_Talk.pdf', 'Post_ASEAN_Presser_2016.pdf', 'Post_G7_Presser_Japan.pdf',
                           'Presser_Unannounced_Heath_Care.pdf', 'Kenya_Civil_Society_Meeting.pdf',
                           'Obama-Romney_-_Second_Live_Debate.pdf', 'Jamaica_Town_Hall.pdf', 'YSEALI_Town_Hall.pdf',
                           'BP_Oil_Spill_Presser.pdf', 'VA_Misconduct_Allegations.pdf', 'ISIL_Pentagon_Update.pdf',
                           'ISIL_Presser_Post_Security_Council_Meeting.pdf']
# maybe: 'Alabama_Tornado_Devestation.pdf'

# the year for Community_College_Plan.pdf is 20105 but should be 2015

Get the path to the directory in which the PDFs are stored.

In [4]:
pdf_dir = Path.cwd().parent / "pdfs"

Filepaths of all the PDFs in the folder `pdf_dir`, can be used to iterate over all the PDFs to store the extracted speeches in a dataframe.

In [5]:
pdfs = list(pdf_dir.glob('*.pdf'))  
print("current number of PDFs:", len(pdfs))

current number of PDFs: 436


Get filepath of the PDF you want to process.

In [6]:
filepath = pdfs[100] 
print("File:", str(filepath.parts[-1]))

File: Senate_Speech_on_Ohio_Electoral_Vote.pdf


Make a PDFHandler object for the given filepath.

In [7]:
pdf = PDFHandler(filepath)

Print the last page of the PDF before it has been processed.

In [8]:
print(pdf.original_page(-1))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
of the country and that these differences turn shamefully on differences of wealth or of race, 
in which too many voters have to contend with state officials, servants of the public, who put 
partisan or personal political interests ahead of the public in administering our elections -- in 
 
such circumstances, we have an obligation to fix the problem.
And I have to add that this is not a problem unique to this election, and it is not a partisan 
problem. Keep in mind, I come from Cook County, from Chicago, in which there is a long 
record of these kinds of problems taking place and disadvantaging Republicans as well as 
Democrats. So I would ask that all of us rise up and use this occasion to amend this problem.  
Thank you. 
Transcription by Michael E. Eidenmuller.        Property of AmericanRhetoric.com                                    Updated 12/16/21        Page 3 


Print the first page of the PDF before it has been processed.

In [9]:
print(pdf.original_page(0))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
B O
arack  bama 
Senate Floor Speech on Ohio Electoral Vote Counting Procedures 
 
Delivered 6 January 2005, Washington, D.C.
 
 
AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio 
 
Thank you very much, Mr. President; Ladies and Gentlemen of the Senate:
I have to say that I didn't anticipate speaking today, but the importance, I think, of this issue 
is one in which I feel it's important for me to address this body.  
 
You know, during the election I had the occasion of meeting a woman who had supported me 
in my campaign. And she decided to come to shake my hand and take a photograph. A 
wonderful woman, she wasn't asking for anything, and I was very grateful that she took time 
to come by. It was a[n] unexceptional moment except for the fact that she was born in 1894. 
And her name was Marguerite Lewis, an African-American woman who had been born in 
Louisiana, born in the shadow of slavery, born at a time 

Define a regular expression to get the date, location, and content of the speech. Extract the entire speech from the PDF.

In [10]:
start = r"^(?:\s*AA +RR\s+mmeerriiccaann +hheettoorriicc\.\.ccoomm|\s*AmericanRhetoric\.com|\s*AAmmeerriiccaannRRhheettoorriicc\.\.ccoomm)"

date = r"(?P<day>[0-9]{1,2})\s+(?P<mon>[a-z]+)\s+(?P<year>[0-9]{2,})"
#loc = r"(?P<location_small>[a-z0-9. ]+),\s+(?P<location_big>[a-z0-9., ]+)"
#loc = r"(?P<location>[a-z0-9.,\- ]+)"
loc = r"(?P<location>.+?)"
auth = r"(?:\[?AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio(\s*and edited for continuity)?\]?)?"

content = r"(?P<content>.*?)"

footnotes = r"(([1-9][0-9]* .+?)+\n+\s*)?"

tr = r"(Transcription\s+by\s+.*?)?"
pr = r"(Property\s+of\s+)?"
end = r"AmericanRhetoric\.com"

full_patern = start + \
r"(\s+.*?(delivered|First +Broadcast)\s+" + date + r"(,\s+" + loc + r")?" + r"\s+" + auth + r")?" + \
r"\s+" + content + r"\n+" + \
r"(?:" + footnotes + tr + r"\s*" + pr + r"\s*" + end + r")"
pat = re.compile(full_patern, re.I | re.DOTALL)

speech = pdf.extract_speech(pat)
print(speech)

Thank you very much, Mr. President; Ladies and Gentlemen of the Senate:
I have to say that I didn't anticipate speaking today, but the importance, I think, of this issue 
is one in which I feel it's important for me to address this body.  
 
You know, during the election I had the occasion of meeting a woman who had supported me 
in my campaign. And she decided to come to shake my hand and take a photograph. A 
wonderful woman, she wasn't asking for anything, and I was very grateful that she took time 
to come by. It was a[n] unexceptional moment except for the fact that she was born in 1894. 
And her name was Marguerite Lewis, an African-American woman who had been born in 
Louisiana, born in the shadow of slavery, born at a time when lynchings were commonplace, 
 
born at a time when African-Americans and women could not vote.
  And yet, over the course of decades she had participated in broadening our democracy and 
ensuring that, in fact, at some point, if not herself, then her chi

Print the relevant info of the PDF.

In [11]:
pdf.print_info()

Title: Senate_Speech_on_Ohio_Electoral_Vote
Number of pages: 3
Date: ['6', 'January', '2005']
Location: ['Washington, D.C.']


Check whether there are multiple speakers. (If there is a high "Obama:" count, this might indicate he is answering questions, i.e., it is an interview, press conference, debate,...)

In [12]:
speakers = pdf.multiple_speakers(speech)
pprint(speakers)

"Obama:" count: 0
"President:" count: 0
"Question:" count: 0
"Audience:" count: 0
"Member:" count: 0
['senate:']


Replace or delete some characters to clean the speech.

In [13]:
old = [r'-+', r'\.{2,}', r'[’‘]', r'"', r'’’', r'‘‘', r'“', r'”', r'\[sic\]', r'\s+', r'\[Source:\s*http.*\]', r'[\[\]\(\)]']
new = [r' ' , r' '     , r"'"   , r'' , r''  , r''  , r'' , r'' , r' '      , r' '  , r'',                     r''      ]

clean_speech = pdf.replace(speech, old, new)
print(clean_speech)

Thank you very much, Mr. President; Ladies and Gentlemen of the Senate: I have to say that I didn't anticipate speaking today, but the importance, I think, of this issue is one in which I feel it's important for me to address this body. You know, during the election I had the occasion of meeting a woman who had supported me in my campaign. And she decided to come to shake my hand and take a photograph. A wonderful woman, she wasn't asking for anything, and I was very grateful that she took time to come by. It was an unexceptional moment except for the fact that she was born in 1894. And her name was Marguerite Lewis, an African American woman who had been born in Louisiana, born in the shadow of slavery, born at a time when lynchings were commonplace, born at a time when African Americans and women could not vote. And yet, over the course of decades she had participated in broadening our democracy and ensuring that, in fact, at some point, if not herself, then her children and her gran